In [12]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)

popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index

melb_df['SellerType'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')

Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:

In [13]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений, тип столбца)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Rooms,9,int64
4,Bathroom,9,int64
5,Car,11,int64
6,Bedroom,12,int64
7,CouncilArea,33,object
8,SellerType,50,object
9,Date,58,datetime64[ns]


Самый простой способ преобразования столбцов к типу данных category — это использование уже знакомого нам метода astype(), в параметры которого достаточно передать строку 'category'.

In [14]:
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          13580 non-null  int64         
 1   Suburb         13580 non-null  object        
 2   Address        13580 non-null  object        
 3   Rooms          13580 non-null  int64         
 4   Type           13580 non-null  object        
 5   Price          13580 non-null  float64       
 6   Method         13580 non-null  object        
 7   SellerG        13580 non-null  object        
 8   Date           13580 non-null  datetime64[ns]
 9   Distance       13580 non-null  float64       
 10  Postcode       13580 non-null  int64         
 11  Bedroom        13580 non-null  int64         
 12  Bathroom       13580 non-null  int64         
 13  Car            13580 non-null  int64         
 14  Landsize       13580 non-null  float64       
 15  BuildingArea   1358

None

Сделаем преобразование столбцов к типу данных category:

In [15]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          13580 non-null  int64         
 1   Suburb         13580 non-null  object        
 2   Address        13580 non-null  object        
 3   Rooms          13580 non-null  int64         
 4   Type           13580 non-null  category      
 5   Price          13580 non-null  float64       
 6   Method         13580 non-null  category      
 7   SellerG        13580 non-null  object        
 8   Date           13580 non-null  datetime64[ns]
 9   Distance       13580 non-null  float64       
 10  Postcode       13580 non-null  int64         
 11  Bedroom        13580 non-null  int64         
 12  Bathroom       13580 non-null  int64         
 13  Car            13580 non-null  int64         
 14  Landsize       13580 non-null  float64       
 15  BuildingArea   1358

None

ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY<br>
У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их.

In [16]:
# с помощью атрибута этого аксессора categories мы можем получить список 
# уникальных категорий в столбце Regionname:

print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


In [17]:
# каким образом столбец кодируется в виде чисел в памяти компьютера

display(melb_df['Regionname'].cat.codes) 

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий. Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.

In [18]:
# переименуем категории признака типа постройки Type — заменим их 
# на полные названия (напомним, u — unit, h — house, t — townhouse).

melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

Создадим объект Series new_houses_types, в котором будем хранить типы зданий новой партии домов. Преобразуем тип new_houses_types в такой же тип, как и у столбца Type в таблице melb_data, и выведем результат на экран:

In [20]:
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

# тип данных category хранит только категории, 
# которые были объявлены при его инициализации.

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

Можно добавить категорию flat в столбец Type с помощью метода акссесора cat add_categories(), в который достаточно просто передать имя новой категории:

In [21]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [22]:
popular_subarb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'].apply(lambda x: x if x in popular_subarb else 'other')

0          Abbotsford
1          Abbotsford
2          Abbotsford
3          Abbotsford
4          Abbotsford
             ...     
13575           other
13576    Williamstown
13577    Williamstown
13578    Williamstown
13579      Yarraville
Name: Suburb, Length: 13580, dtype: object

In [23]:
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          13580 non-null  int64         
 1   Suburb         13580 non-null  category      
 2   Address        13580 non-null  object        
 3   Rooms          13580 non-null  int64         
 4   Type           13580 non-null  category      
 5   Price          13580 non-null  float64       
 6   Method         13580 non-null  category      
 7   SellerG        13580 non-null  object        
 8   Date           13580 non-null  datetime64[ns]
 9   Distance       13580 non-null  float64       
 10  Postcode       13580 non-null  int64         
 11  Bedroom        13580 non-null  int64         
 12  Bathroom       13580 non-null  int64         
 13  Car            13580 non-null  int64         
 14  Landsize       13580 non-null  float64       
 15  BuildingArea   1358

None